1. sqlite
2. MySQL
3. SQLAlchemy
4. MongoDB
5. Pandas
6. Matplotlib
7. Plotly

In [95]:
!pip install PyMySQL

    100% |████████████████████████████████| 81kB 293kB/s ta 0:00:01


In [1]:
import sqlite3
import datetime

In [2]:
# 建立一個sqlite連線
# conn = sqlite3.connect('ptt.db', isolation_level=None) => auto commit mode
conn = sqlite3.connect('ptt.db')
conn.row_factory = sqlite3.Row
c = conn.cursor()

In [37]:
c.execute("""DROP TABLE articles""")

0

In [4]:
# 執行SQL
c.execute("""CREATE TABLE articles
             (url VARCHAR(255) PRIMARY KEY,
             dt TIMESTAMP,
             title VARCHAR(255),
             content TEXT,
             board VARCHAR(255))""")

In [5]:
data = {
    'url': 'https://ptt.cc',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩',
    'board': 'Python'
}
c.execute("""INSERT INTO articles VALUES (:url, :dt, :title, :content, :board)""", data)

In [6]:
data = [{
    'url': 'https://ptt.cc/1',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩!!!!!!!!',
    'board': 'Python'
},{
    'url': 'https://ptt.cc/2',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩 :) :)',
    'board': 'Python'
},{
    'url': 'https://ptt.cc/3',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩 yea',
    'board': 'Python'
}]
    
c.executemany("""INSERT OR REPLACE INTO articles VALUES (:url, :dt, :title, :content, :board)""", data)

In [7]:
conn.in_transaction

True

In [8]:
conn.commit()

In [9]:
conn.in_transaction

False

In [10]:
c.execute("""CREATE INDEX dt_idx ON articles (dt)""")

In [11]:
conn.commit()

In [12]:
cur = c.execute("""SELECT * FROM articles""")

In [13]:
for row in cur:
    print(type(row))
    print(row['url'])

<class 'sqlite3.Row'>
https://ptt.cc
<class 'sqlite3.Row'>
https://ptt.cc/1
<class 'sqlite3.Row'>
https://ptt.cc/2
<class 'sqlite3.Row'>
https://ptt.cc/3


In [14]:
#cur = conn.execute("""SELECT * FROM articles WHERE dt > ?""", tuple([1490194193]))
cur = c.execute("""SELECT * FROM articles WHERE dt > ?""", (1490194193,))

for row in cur:
    print(row['url'])

https://ptt.cc
https://ptt.cc/1
https://ptt.cc/2
https://ptt.cc/3


In [15]:
c.close()

In [16]:
c.execute("SELECT * FROM articles")

ProgrammingError: Cannot operate on a closed cursor.

In [17]:
import pymysql

conn = pymysql.connect(host='localhost',
                             user='root',
                             db='ab106',
                             charset='utf8',
                             autocommit=True,
                             cursorclass=pymysql.cursors.DictCursor)
c = conn.cursor()

In [21]:
c.execute("""DROP TABLE IF EXISTS articles""")

/Users/ian/.pyenv/versions/3.6.0/lib/python3.6/site-packages/pymysql/cursors.py:323: Warning: (1051, "Unknown table 'ab106.articles'")
  self._do_get_result()


0

In [22]:
# 執行SQL
c.execute("""CREATE TABLE articles
             (url VARCHAR(255) PRIMARY KEY,
             dt TIMESTAMP,
             title VARCHAR(255),
             content TEXT,
             board VARCHAR(255))""")

0

In [23]:
data = {
    'url': 'https://ptt.cc',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩',
    'board': 'Python'
}
c.execute("""INSERT INTO articles VALUES (%(url)s, %(dt)s, %(title)s, %(content)s, %(board)s)""", data)

1

In [29]:
data = [{
    'url': 'https://ptt.cc/1',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩!!!!!!!!',
    'board': 'Python'
},{
    'url': 'https://ptt.cc/2',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩 :) :) XD',
    'board': 'Python'
},{
    'url': 'https://ptt.cc/3',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩 yea',
    'board': 'Python'
}]
    
c.executemany("""REPLACE INTO articles VALUES (%(url)s, %(dt)s, %(title)s, %(content)s, %(board)s)""", data)
# c.executemany("""INSERT INTO articles
#                  VALUES
#                    (%(url)s, %(dt)s, %(title)s, %(content)s, %(board)s)
#                  ON DUPLICATE KEY UPDATE
#                    (url = %(url)s,
#                    dt = %(dt)s,
#                    title = %(title)s,
#                    content = %(content)s,
#                    board = %(board)s)""", data)

6

In [36]:
c.execute("""SELECT * FROM articles WHERE dt > %s""", (datetime.datetime.now() - datetime.timedelta(seconds=500),))

for row in c:
    print(row['url'])

https://ptt.cc/1
https://ptt.cc/2
https://ptt.cc/3


In [38]:
!pip install sqlalchemy

In [46]:
from sqlalchemy import Column, Integer, String, DateTime, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
 
Base = declarative_base()
 
class Article(Base):
    __tablename__ = 'article'

    url = Column(String(255), primary_key=True)
    title = Column(String(255), nullable=False)
    dt = Column(DateTime)
    content = Column(Text)
    board = Column(String(255))

engine = create_engine('mysql://root@localhost/ab106')
 
Base.metadata.create_all(engine)

In [47]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

Base.metadata.bind = engine
 
DBSession = sessionmaker(bind=engine)

session = DBSession()

data = {
    'url': 'https://ptt.cc',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩',
    'board': 'Python'
}
new_article = Article(**data)
session.add(new_article)
session.commit()

In [55]:
data = [{
    'url': 'https://ptt.cc/1',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩!!!!!!!!',
    'board': 'Python'
},{
    'url': 'https://ptt.cc/2',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩 :) :) XD',
    'board': 'Python'
},{
    'url': 'https://ptt.cc/3',
    'dt': datetime.datetime.now(),
    'title': '爆卦',
    'content': 'Python很好玩 yea',
    'board': 'Python'
}]
session.bulk_insert_mappings(Article, data)

In [56]:
session.commit()

In [53]:
cur = session.query(Article).filter(Article.url == 'https://ptt.cc')

In [54]:
for row in cur:
    print(row.url)

https://ptt.cc


In [11]:
import pandas as pd

In [14]:
df = pd.read_sql('SELECT * FROM articles', conn)

In [15]:
df.head()

,url,dt,title,content,board
0,https://ptt.cc,1490192171,爆卦,Python很好玩,Python
1,https://ptt.cc/1,1490192179,爆卦,Python很好玩!!!!!!!!,Python
2,https://ptt.cc/2,1490192179,爆卦,Python很好玩 :) :),Python
3,https://ptt.cc/3,1490192179,爆卦,Python很好玩 yea,Python


In [16]:
df.query('dt > 1490192178')

,url,dt,title,content,board
1,https://ptt.cc/1,1490192179,爆卦,Python很好玩!!!!!!!!,Python
2,https://ptt.cc/2,1490192179,爆卦,Python很好玩 :) :),Python
3,https://ptt.cc/3,1490192179,爆卦,Python很好玩 yea,Python


In [20]:
df.dt.sum()

5960768708

In [25]:
df = pd.read_csv('./regional_water_consumption.csv', encoding='big5')

In [27]:
df.head()

,收費年月,一級發布區代碼,用水種別,用水量
0,201601,A0201-01-006,8,502
1,201601,A0201-03-002,1,4533
2,201601,A0201-03-002,2,1550
3,201601,A0201-03-003,1,126
4,201601,A0201-03-008,8,325


In [28]:
df.shape

(385983, 4)

In [30]:
df2 = pd.read_csv('./regional_water_consumption_201607-201612.csv', encoding='big5')

In [31]:
df2.head()

,收費年月,一級發布區代碼,用水種別,用水量
0,201607,A0201-03-002,1,4400
1,201607,A0201-03-003,1,119
2,201607,A0201-03-009,1,93
3,201607,A0201-05-004,1,1
4,201607,A0201-06-001,1,3457


In [34]:
df2.shape

(388046, 4)

In [35]:
df3 = df.append(df2)

In [36]:
df3.shape

(774029, 4)

In [37]:
df3.head()

,收費年月,一級發布區代碼,用水種別,用水量
0,201601,A0201-01-006,8,502
1,201601,A0201-03-002,1,4533
2,201601,A0201-03-002,2,1550
3,201601,A0201-03-003,1,126
4,201601,A0201-03-008,8,325


In [38]:
df3.tail()

,收費年月,一級發布區代碼,用水種別,用水量
388041,201612,A6812-21-007,9,17
388042,201612,A6812-21-008,9,8
388043,201612,A6812-21-010,9,55
388044,201612,A6812-21-011,9,4
388045,201612,A6812-21-012,9,268


In [39]:
df3.to_sql('water', conn)

In [40]:
df3['用水量'].max()

5569676

In [41]:
df3['用水量'].min()

0

In [42]:
df3['用水量'].mean()

2559.7146605618136

In [43]:
df3[df3['用水量'] == df3['用水量'].min()]

,收費年月,一級發布區代碼,用水種別,用水量
16,201601,A0201-06-003,2,0
20,201601,A0201-06-004,9,0
71,201601,A0201-09-007,2,0
72,201601,A0201-09-007,9,0
137,201601,A0201-13-009,9,0
193,201601,A0201-18-007,9,0
225,201601,A0201-20-008,4,0
249,201601,A0201-23-002,9,0
392,201601,A0201-33-004,4,0
434,201601,A0201-36-003,4,0


In [44]:
df3[df3['用水量'] == df3['用水量'].max()]

,收費年月,一級發布區代碼,用水種別,用水量
45120,201607,A6411-69-006,3,5569676


In [65]:
df3.loc[:,['收費年月', '用水量']]

,收費年月,用水量
0,201601,502
1,201601,4533
2,201601,1550
3,201601,126
4,201601,325
5,201601,88
6,201601,1
7,201601,3474
8,201601,196
9,201601,47


In [66]:
df3.loc[:,['收費年月', '用水量']].groupby('收費年月').sum()

,用水量
收費年月,
201601,160276683
201602,148401113
201603,143786708
201604,155289443
201605,161298493
201606,165722774
201607,175637322
201608,172619609
201609,181399482
